In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### 1.TFRecord简介

TFRecord是Google官方推荐的一种数据格式，是Google专门为TensorFlow设计的一种数据格式。

实际上，TFRecord是一种二进制文件，其能更好的利用内存，其内部包含了多个**tf.train.Example**，而Example是protocol buffer数据标准的实现，在一个Example消息体中包含了一系列的tf.train.feature属性，而每一个feature是一个key-value的键值对，其中，key是string类型，而value的取值有三种：

- bytes_list:可以存储string 和byte两种数据类型
- float_list:可以存储float(float32) 和 double(float64)两种数据类型。
- int64_list:可以存储：bool,enum,int32,uint32,int64,uint64

- int64_list：tf.train.Feature(int64_list = tf.train.int64List(value = 输入))

- float_list:tf.train.Feature(float_list = tf.train.FloatList(value = 输入))

- bytes_list:tf.train.Feature(bytes_list = tf.train.BytesList(value = 输入))

注意：输入必须是list（向量）

值得一提的是，TensorFlow源码中到处可见.proto的文件，且这些文件定义了TensorFlow重要的数据结构部分，且多重语言可以直接使用这类数据，很强大。

##### 为什么用TFRcord

TFRcord并非是TensorFlow唯一支持的数据格式，你也可以使用CSV或文本等格式，但是对于TensorFlow来说，TFRcord是最友好的，也是最方便的。前面提到，TFRcord内部是一系列实现了protocol buffer数据标准的Example。对于大型数据，像比其余数据格式，protocol buffer类型的数据优势很明显。_

在数据集较小时，我们会把数据全部加载到内存里方便快速导入，但当数据量超过内存大小时，就只能放在硬盘上来一点点读取，这时就不得不考虑数据的移动、读取、处理等速度。使用TFRecord就是为了提速和节约空间的。

![TFRcord格式理解](./markdown_pics/TFRcord格式理解.png)

![TFRcord格式理解](./markdown_pics/TFRcord格式理解2.png)

#### 2.写入TFRecord文件


![TFRcord格式理解](./markdown_pics/生成TFRecord格式数据.png)

#### 3.读取TFRecord文件

![读取TFRecord格式数据](./markdown_pics/读取TFRecord格式数据.png)

#### 4.案例详解


In [49]:
os.chdir('/home/hp/git/learningzone/huangpei/eat_tensorflow2_in_30_days/')
data_dir = './data/datasets'
train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
test_cats_dir = data_dir + '/valid/cats/'
test_dogs_dir = data_dir + '/valid/dogs/' 

train_tfrecord_file = data_dir + '/train/train.tfrecords'
test_tfrecord_file = data_dir + '/valid/test.tfrecords'

##### 对训练集执行读取

In [50]:
train_cat_filenames = [train_cats_dir + filename for filename in os.listdir(train_cats_dir)]

train_dog_filenames = [train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)]

train_filenames = train_cat_filenames + train_dog_filenames


# 对数据集配置标签 cat 0 dog 1
train_labels = [0] * len(train_cat_filenames) + [1] * len(train_dog_filenames)

In [51]:
with tf.io.TFRecordWriter(train_tfrecord_file) as writer:
    for filename, label in zip(train_filenames, train_labels):

        image = open(filename, 'rb').read()  # 读取数据集图片到内存， image为一个byte类型的字符串

        feature = {   # 建立一个tf.train.Feature的字典
            'image':tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])), # 图片是一个Bytes对象
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value = [label]))  # 标签是一个Int对象
        }
        example = tf.train.Example(features = tf.train.Features(feature = feature))  # 通过字典建立Example

        writer.write(example.SerializePartialToString())  # 将Example 序列化并写入TFrecord文件  

##### 对验证集执行读取

In [52]:
test_cat_filenames = [test_cats_dir + filename for filename in os.listdir(test_cats_dir)]

test_dog_filenames = [test_dogs_dir + filename for filename in os.listdir(test_dogs_dir)]

test_filenames = test_cat_filenames + test_dog_filenames


# 对数据集配置标签 cat 0 dog 1
test_labels = [0] * len(test_cat_filenames) + [1] * len(test_dog_filenames)

In [53]:
with tf.io.TFRecordWriter(test_tfrecord_file) as writer:
    for filename, label in zip(test_filenames, test_labels):

        image = open(filename, 'rb').read()  # 读取数据集图片到内存， image为一个byte类型的字符串

        feature = {   # 建立一个tf.test.Feature的字典
            'image':tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])), # 图片是一个Bytes对象
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value = [label]))  # 标签是一个Int对象
        }
        example = tf.train.Example(features = tf.train.Features(feature = feature))  # 通过字典建立Example

        serialized = example.SerializePartialToString()  # 将Example序列化
        writer.write(serialized)  # 写入TFrecord文件

#### 读取TFRecord文件

In [54]:
feature_description = { # 定义 feature结构，告诉解码器每个Feature的类型是什么
    'image':tf.io.FixedLenFeature([], tf.string),
    'label':tf.io.FixedLenFeature([], tf.int64),

}


def _parse_example(example_string): # 将TFRecord 文件中的每一个序列化的 tf.train.Example 解码
    feature_dict = tf.io.parse_single_example(example_string, feature_description)

    feature_dict['image'] = tf.io.decode_jpeg(feature_dict['image'])  # 解码JPEG图片

    feature_dict['image'] = tf.image.resize(feature_dict['image'], [256,256]) /255.0

    return feature_dict['image'], feature_dict['label']

In [59]:
batch_size = 32


#### 训练集读取
train_dataset = tf.data.TFRecordDataset(train_tfrecord_file) # 读取 TFRecord文件

train_dataset = train_dataset.map(_parse_example)
train_dataset = train_dataset.shuffle(buffer_size = 23000)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)



#### 测试集读取

test_dataset = tf.data.TFRecordDataset(test_tfrecord_file)  # 读取TFRecord文件

test_dataset = test_dataset.map(_parse_example)
test_dataset = test_dataset.batch(batch_size)

In [60]:
# 设计神经网络模型,卷积和池化
class CNNModel(tf.keras.models.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32,3, activation= 'relu')
        self.maxpool1 = tf.keras.layers.MaxPooling2D()
        self.conv2 = tf.keras.layers.Conv2D(32,5,activation='relu')
        self.maxpool2 = tf.keras.layers.MaxPooling2D()
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(64,activation='relu')
        self.d2 = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

In [61]:
learning_rate = 0.001
model = CNNModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [62]:
EPOCHS = 10
for epoch in range(EPOCHS):
    # 在下一个epoch开始的时候，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()

    test_loss.reset_states()
    test_accuracy.reset_states()
    

    for images, labels in train_dataset:
        train_step(images, labels) 


    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels) 

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))

Epoch 1, Loss: 0.6903296709060669, Accuracy: 56.017391204833984, Test Loss: 0.644851803779602, Test Accuracy: 63.55000305175781
Epoch 2, Loss: 0.6089661121368408, Accuracy: 67.38260650634766, Test Loss: 0.6182286739349365, Test Accuracy: 67.54999542236328
Epoch 3, Loss: 0.5100833177566528, Accuracy: 74.71304321289062, Test Loss: 0.5957335233688354, Test Accuracy: 69.19999694824219
Epoch 4, Loss: 0.3423081934452057, Accuracy: 84.91304779052734, Test Loss: 0.7056066393852234, Test Accuracy: 68.8499984741211
Epoch 5, Loss: 0.13823211193084717, Accuracy: 94.9000015258789, Test Loss: 1.0297844409942627, Test Accuracy: 69.0999984741211
Epoch 6, Loss: 0.03975549340248108, Accuracy: 98.81304168701172, Test Loss: 1.6096988916397095, Test Accuracy: 67.75
Epoch 7, Loss: 0.01992863230407238, Accuracy: 99.47391510009766, Test Loss: 1.7852365970611572, Test Accuracy: 67.8499984741211
Epoch 8, Loss: 0.017125653102993965, Accuracy: 99.55652618408203, Test Loss: 2.081528425216675, Test Accuracy: 68.550